<a href="https://colab.research.google.com/github/useruser0012/statistics_festival/blob/main/%EC%84%A4%EB%AC%B8_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials

MAX_TOKENS = 10  # 총 코인 수
COST_PER_PLAY = 2  # 한 번 플레이에 드는 코인

def save_to_google_sheet(data):
    # 구글 API 인증 및 시트 열기
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/vnfna/Downloads/statistics-festival-91c4b903722e.json', scope)
    client = gspread.authorize(creds)

    sheet = client.open("통계활용페스티벌").sheet1

    # 저장할 데이터 (한 줄)
    row = [
        data.get('name',''),
        data.get('group_code', ''),
        data.get('total_plays', ''),
        data.get('success_count', ''),
        data.get('failure_count', ''),
        data.get('more_try', ''),
        data.get('expectation', ''),
        data.get('emotion', ''),
        data.get('much_more_try', ''),
        data.get('lack', ''),
        data.get('situation', '')
    ]

    sheet.append_row(row)
    print("설문 및 게임 결과가 구글 스프레드시트에 저장되었습니다.")

def play_game(group):
    if group == "1반":  # 성공 그룹
        return "성공"
    elif group == "4반":  # 실패 그룹
        return "실패"
    else:  # 2반, 3반 = 랜덤 그룹
        return random.choices(["성공", "실패"], weights=[0.3, 0.7])[0]

def main():
    print('\n 통계 활용 페스티벌을 위한 간단한 설문조사입니다. \n 프로그램 진행 방식에 잘 따라주세요!')
    print('\n 진행 방식은 다음과 같습니다. \n1. 처음은 코인 10개로 시작합니다.')
    print('2. 한 번 도전할 때마다 코인이 2개씩 소모됩니다.')
    print('3. 결과는 \'성공\' 또는 \'실패\'로 나뉘며, 성공 시 코인을 보상으로 얻습니다.')
    print('4. 코인은 0개가 되도 게임이 계속되며, 그만하고 싶을 때에는 \'q\'를 입력하시면 됩니다.')
    print(' 게임이 끝난 후에는 간단한 설문이 준비되어있습니다. 그럼 시작할까요?')
    name = input(' 이름을 입력해주세요. 별명도 괜찮습니다!: ').strip()
    print("반을 숫자만 입력해주세요.:")
    print("1반\n2반\n3반\n4반")
    group_input = input("반 입력: ").strip()

    if group_input == "1":
        group = "1반"
        group_code = 1
    elif group_input == "2":
        group = "2반"
        group_code = 2
    elif group_input == "3":
        group = "3반"
        group_code = 3
    elif group_input == "4":
        group = "4반"
        group_code = 4
    else:
        print("잘못된 입력입니다. 프로그램을 종료합니다.")
        return

    print(f"\n당신은 '{group}'에 속했습니다.")
    tokens = MAX_TOKENS
    results = []

    # 코인이 음수여도 게임 계속 진행
    while True:
        print(f"\n현재 코인: {tokens}")
        user_input = input("도전하려면 Enter, 그만하려면 'q'를 입력하세요: ").strip()
        if user_input.lower() == 'q':
            break

        tokens -= COST_PER_PLAY
        result = play_game(group)
        results.append(result)

        if result == "성공":
            reward = random.choice([3, 4])
            tokens += reward
            print(f"✨ 성공! 보상으로 코인 {reward}개를 획득했습니다! (총 코인: {tokens})")
        else:
            print("💥 실패! 보상은 없습니다.")

    print("\n--- 실험 종료 ---")
    print(f"총 도전 횟수: {len(results)}")
    print(f"성공 횟수: {results.count('성공')}")
    print(f"실패 횟수: {results.count('실패')}")

    # 설문 시작 안내
    print('\n이제 간단한 설문을 진행하겠습니다. \n설문이 모두 서술이라 양해 부탁드립니다 ㅠ')
    print('*\'표시가 된 설문만큼은 꼭 작성해주세요!!')

    # 간단한 설문
    more_try = input("\n*더 도전하고 싶은 마음(충동)이 있었나요? (예/아니오): ")
    much_more_try = input('있었다면 왜 그런 마음(충동)이 있었는지 이유를 적어주세요.')
    expectation = input("*성공할 것 같은 기대감은 어느 정도였나요? (0~100): ")
    emotion = input("*도전 중 어떤 기분을 느꼈나요?: ")
    lack = input('코인이 부족할 때 어떤 생각이 들었나요?: ')
    situation = input('이 게임은 어떤 실제 상황과 비슷하다고 느끼셨나요?: ')

    data_to_save = {
        'name': name,
        'group_code': group_code,
        'total_plays': len(results),
        'success_count': results.count('성공'),
        'failure_count': results.count('실패'),
        'more_try': more_try,
        'much_more_try': much_more_try,
        'expectation': expectation,
        'emotion': emotion,
        'lack': lack,
        'situation': situation
    }

    save_answer = input("\n설문이 완료되었습니다. 참여해주셔서 감사합니다! \n아무거나 눌러주세요:)")
    try:
        save_to_google_sheet(data_to_save)
    except Exception as e:
        print("구글 스프레드시트 저장 중 오류 발생:", e)


    print("\n프로그램이 종료되었습니다.")

def save_to_google_sheet(data):
    # 구글 API 인증 및 시트 열기
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/vnfna/Downloads/statistics-festival-91c4b903722e.json', scope)
    client = gspread.authorize(creds)

    sheet = client.open("통계활용페스티벌").sheet1

    # 저장할 데이터 (한 줄)
    row = [
        data.get('name', ''),
        data.get('group_code', ''),
        data.get('total_plays', ''),
        data.get('success_count', ''),
        data.get('failure_count', ''),
        data.get('more_try', ''),
        data.get('expectation', ''),
        data.get('emotion', ''),
        data.get('much_more_try', ''),
        data.get('lack', ''),
        data.get('situation', '')
    ]

    sheet.append_row(row)
    print("설문 및 게임 결과가 저장되었습니다.")

if __name__ == "__main__":
    main()



